In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [7]:

main_webUrl = "https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=500000000&minPrice=100000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [8]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [9]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

197

In [10]:
dataframe_low = {}
dataframe_low["Nama"] = []
dataframe_low["Harga"] = []

### Low 100M - 500M

In [11]:
for i in range(1, 40):
    webUrl = f"https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=500000000&minPrice=100000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_low["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_low["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_low.keys():
                dataframe_low[prop_key] = []
            
            if prop_key in dataframe_low.keys():
                dataframe_low[prop_key].append(prop_value)
                
            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/sleman/hos19336399/ Rp 411 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 73 m²
Luas Bangunan: 38 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/yogyakarta/hos19328645/ Rp 411 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 72 m²
Luas Bangunan: 38 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/sleman/hos18934735/ Rp 500 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 125 m²
Luas Bangunan: 45 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Jumlah Lantai: 1
Kondisi Properti: Baru

1
/properti/bantul/hos16805466/ Rp 350 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 87 m²
Luas Bangunan: 36 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 1
Kondisi Properti: Baru

1
/properti/bantul/hos18460951/ Rp 350 Juta
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 314 m²
Luas Bangunan: 100 m²
Sertifikat: SHM
Daya Listrik: Lainnya

1
/properti

In [12]:
rumah123_df_low = pd.DataFrame.from_dict(dataframe_low, orient='index').T
rumah123_df_low = rumah123_df_low.fillna(0)
rumah123_df_low.to_csv("Yogya_low.csv", index=False)

In [14]:
rumah123_df_low

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Garasi,Kamar Tidur Pembantu,Kamar Mandi Pembantu
0,"Rumah Murah SHM, Bisa KPR, Kawasan Perumahan, ...",Rp 411 Juta,2,1,73 m²,38 m²,1,SHM,1300 Watt,1,Bagus,1,1,1
1,"Rumah Baru SHM, Bisa KPR dalam Kawasan Perumah...",Rp 411 Juta,2,1,72 m²,38 m²,1,SHM,1300 Watt,1,Bagus,1,1,1
2,Kami Hadir Dengan Harga Lebih Terjangkau Di Sl...,Rp 500 Juta,2,1,125 m²,45 m²,1,SHM,1300 Watt,1,Baru,1,1,1
3,Promo Romadhon Dan Lebaran 350 Juta Saja Free ...,Rp 350 Juta,2,1,87 m²,36 m²,1,SHM,Lainnya,1,Baru,1,1,1
4,"Rumah Murah Aset BRI, Imogiri, Kab. Bantul",Rp 350 Juta,3,2,314 m²,100 m²,1,SHM,Lainnya,1,Bagus,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,RUMAH CANTIK BERKUALITAS STRATEGIS 10 MENIT BA...,Rp 300 Juta,2,1,80 m²,45 m²,0,SHM,1300 Watt,0,0,0,0,0
209,"Rumah Skandinavian di Bantul, Dekat Kota Jogja...",Rp 245 Juta,2,1,83 m²,36 m²,0,SHM,900 Watt,0,0,0,0,0
210,Rumah Dekat Pusat Kota Jogja,Rp 375 Juta,0,1,98 m²,40 m²,0,SHM,900 Watt,0,0,0,0,0
211,"Rumah Type 40, Lokasi Strategis Dekat Fasum, B...",Rp 375 Juta,0,0,63 m²,40 m²,0,SHM,900 Watt,0,0,0,0,0


In [ ]:
dataframe_low

### Low-Mid 500M-1B

In [15]:
main_webUrl = 'https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=1000000000&minPrice=500000000&page=1'

req_main = requests.get(main_webUrl, headers=headers)

In [16]:
main_html = BeautifulSoup(req_main.content, 'html5lib')

In [17]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

330

In [20]:
dataframe_low_mid = {}
dataframe_low_mid["Nama"] = []
dataframe_low_mid["Harga"] = []

In [21]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=1000000000&minPrice=500000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_low_mid["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_low_mid["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_low_mid.keys():
                dataframe_low_mid[prop_key] = []
            
            if prop_key in dataframe_low_mid.keys():
                dataframe_low_mid[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/sleman/hos18631640/ Rp 750 Juta
Kamar Tidur: 3
Kamar Mandi: 1
Luas Tanah: 210 m²
Luas Bangunan: 170 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 1300 Watt
Garasi: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos19372641/ Rp 660 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 85 m²
Luas Bangunan: 60 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/sleman/hos18934735/ Rp 500 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 125 m²
Luas Bangunan: 45 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Jumlah Lantai: 1
Kondisi Properti: Baru

1
/properti/sleman/hos18228650/  
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 106 m²
Luas Bangunan: 80 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 1300 Watt
Garasi: 1
Kondisi Properti: Baru

1
/properti/sleman/hos19075740/ Rp 768 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 104 m²
Luas Bangunan: 60 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 1
Kondisi 

In [22]:
df_low_mid = pd.DataFrame.from_dict(dataframe_low_mid, orient='index').T
df_low_mid = df_low_mid.fillna(0)
df_low_mid['Category'] = 'Low-Mid'
df_low_mid.to_csv('Yogya_low_mid.csv', index=False)

In [23]:
df_low_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Garasi,Kondisi Properti,Jumlah Lantai,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,Rumah Besar Murah !! Tanahnya 210m Lho..carpot...,Rp 750 Juta,3,1,210 m²,170 m²,2,SHM,1300 Watt,2,Bagus,1,1,1,Low-Mid
1,Rumah Minimalis Cantik Dengan Keindahan View P...,Rp 660 Juta,2,1,85 m²,60 m²,1,SHM,Lainnya,1,Bagus,1,1,1,Low-Mid
2,Kami Hadir Dengan Harga Lebih Terjangkau Di Sl...,Rp 500 Juta,2,1,125 m²,45 m²,1,SHM,1300 Watt,1,Baru,1,1,1,Low-Mid
3,Rumah Baru Cantik Mewah - Harga Super Murah Di...,,3,2,106 m²,80 m²,1,SHM,1300 Watt,1,Baru,1,0,0,Low-Mid
4,Perumahan One Gate Sistem Yang Bikin Aman Dan ...,Rp 768 Juta,2,1,104 m²,60 m²,1,SHM,2200 Watt,1,Baru,1,0,0,Low-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Hunian Modern Baru di Wonosalam Sukoharjo Ngak...,Rp 855 Juta,2,2,107 m²,71 m²,0,SHM,Lainnya,0,0,0,0,0,Low-Mid
223,Hunian Modern Baru di Dekat Klidon Sukoharjo N...,Rp 850 Juta,3,2,127 m²,90 m²,0,SHM,1300 Watt,0,0,0,0,0,Low-Mid
224,Murah Istimewa!!! Rumah Baru Mewah Limasan Mod...,Rp 810 Juta,3,2,125 m²,85 m²,0,SHM,1300 Watt,0,0,0,0,0,Low-Mid
225,Rumah Modern Minimalis Baru di Timur Kampus UI...,Rp 810 Juta,3,2,110 m²,70 m²,0,SHM,1300 Watt,0,0,0,0,0,Low-Mid


### Up-Mid 1B-3B

In [24]:
main_webUrl = "https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=3000000000&minPrice=1000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [25]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [26]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

491

In [27]:
dataframe_up_mid = {}
dataframe_up_mid["Nama"] = []
dataframe_up_mid["Harga"] = []

In [28]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=3000000000&minPrice=1000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_up_mid["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_up_mid["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_up_mid.keys():
                dataframe_up_mid[prop_key] = []
            
            if prop_key in dataframe_low_mid.keys():
                dataframe_up_mid[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/sleman/hos19398054/  
Kamar Tidur: 4
Kamar Mandi: 2
Luas Tanah: 128 m²
Luas Bangunan: 97 m²
Carport: 1
Sertifikat: HGB
Daya Listrik: 5500 Watt
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos38833327/ Rp 1,5 Miliar
Kamar Tidur: 4
Kamar Mandi: 4
Luas Tanah: 150 m²
Luas Bangunan: 150 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 2
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos19552847/ Rp 2,2 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 130 m²
Luas Bangunan: 125 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos19462273/ Rp 1,28 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 147 m²
Luas Bangunan: 95 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/sleman/hos38840914/ Rp 1,47 Miliar
Kamar Tidur: 3
Kamar Mandi: 3
Luas Tanah: 124 m²
Luas Bangunan: 80 m²
Sertifikat: SHM
Daya Listrik: 2200 Watt


In [29]:
df_up_mid = pd.DataFrame.from_dict(dataframe_up_mid, orient='index').T
df_up_mid = df_up_mid.fillna(0)
df_up_mid['Category'] = 'Up-Mid'
df_up_mid.to_csv('Yogya_Up_mid.csv', index=False)

In [30]:
df_up_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Garasi,Jumlah Lantai,Kondisi Properti,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,Rumah Cantik Dekat Ugm!! Lingkungan Nyaman,,4,2,128 m²,97 m²,1,HGB,5500 Watt,1,2,Bagus,1,1,Up-Mid
1,Banting Harga! Rumah Mewah Modern 2 Lantai Ber...,"Rp 1,5 Miliar",4,4,150 m²,150 m²,2,SHM,Lainnya,2,2,Bagus,1,1,Up-Mid
2,Rumah Siap Huni Dekat Upn Di Babarsari Sleman ...,"Rp 2,2 Miliar",3,2,130 m²,125 m²,1,SHM,2200 Watt,1,2,Bagus,1,1,Up-Mid
3,Rumah Mewah,"Rp 1,28 Miliar",3,2,147 m²,95 m²,2,SHM,Lainnya,1,1,Bagus,1,1,Up-Mid
4,HUNIAN MODERN DENGAN LOKASI STRATEGIS,"Rp 1,47 Miliar",3,3,124 m²,80 m²,2,SHM,2200 Watt,2,1,Bagus,1,1,Up-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Kosan Berikut Rumah Induk Dekat Kampus Di Nolo...,"Rp 1,7 Miliar",5,4,245 m²,200 m²,0,SHM,Lainnya,0,0,0,0,0,Up-Mid
234,Dijual Rumah Satrategis Posisi Hok Dalam Kota ...,"Rp 1,5 Miliar",5,3,150 m²,336 m²,0,SHM,Lainnya,0,0,0,0,0,Up-Mid
235,Rumah 2 Lantai dengan Rooftop dan Furnished In...,"Rp 2,25 Miliar",4,3,130 m²,120 m²,0,SHM,Lainnya,0,0,0,0,0,Up-Mid
236,Rumah Furnished Modern dengan Bonus Lengkap di...,"Rp 2,1 Miliar",4,4,144 m²,150 m²,0,SHM,Lainnya,0,0,0,0,0,Up-Mid


### High 3B-7B

In [31]:
main_webUrl = "https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=7000000000&minPrice=3000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [32]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [33]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

153

In [34]:
dataframe_high = {}
dataframe_high["Nama"] = []
dataframe_high["Harga"] = []

In [35]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=7000000000&minPrice=3000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_high["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_high["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_high.keys():
                dataframe_high[prop_key] = []
            
            if prop_key in dataframe_high.keys():
                dataframe_high[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/yogyakarta/hos38830874/ Rp 4,6 Miliar
Kamar Tidur: 6
Kamar Mandi: 3
Luas Tanah: 420 m²
Luas Bangunan: 280 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 5500 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/yogyakarta/hos19333156/ Rp 3 Miliar
Kamar Tidur: 5
Kamar Mandi: 5
Luas Tanah: 200 m²
Luas Bangunan: 255 m²
Carport: 2
Sertifikat: HGB
Daya Listrik: Lainnya
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos19583271/ Rp 4,9 Miliar
Kamar Tidur: 4
Kamar Mandi: 4
Luas Tanah: 193 m²
Luas Bangunan: 300 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/sleman/hos19243170/ Rp 3,5 Miliar
Kamar Mandi: 3
Luas Tanah: 457 m²
Luas Bangunan: 150 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 1

1
/properti/yogyakarta/hos14460371/ Rp 3,5 Miliar
Kamar Tidur: 5
Kamar Mandi: 3
Luas Tanah: 291 m²
Luas Bangunan: 250 m²
Sertifikat: SHM
Daya

In [38]:
df_high = pd.DataFrame.from_dict(dataframe_high, orient='index').T
df_high = df_high.fillna(0)
df_high['Category'] = 'High'
df_high.to_csv('Yogya_high.csv', index=False)

In [39]:
df_high

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Jumlah Lantai,Kondisi Properti,Category
0,Rumah 2 Lantai Dengan Lokasi Sangat Strategis,"Rp 4,6 Miliar",6,3,420 m²,280 m²,2,SHM,5500 Watt,1,1,1,2,Bagus,High
1,Rumah Mewah Tropis 2 Lantai Full Furnished Dal...,Rp 3 Miliar,5,5,200 m²,255 m²,2,HGB,Lainnya,1,1,2,2,Bagus,High
2,Rumah Mewah Premium 2 Lantai Dalam Perumahan M...,"Rp 4,9 Miliar",4,4,193 m²,300 m²,1,SHM,Lainnya,1,1,7,2,Bagus,High
3,Rumah Villa Cantik Modern Di Jakal Kaliurang K...,"Rp 3,5 Miliar",5,3,457 m²,150 m²,2,SHM,Lainnya,1,1,1,1,Butuh Renovasi,High
4,Rumah pogung baru dekat UGM Jogja & RS Sardjito,"Rp 3,5 Miliar",19,3,291 m²,250 m²,2,SHM,Lainnya,1,1,2,2,Bagus,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,Perumahan One Gate System Fasilitas Perumahan ...,"Rp 3,2 Miliar",3,7,186 m²,200 m²,0,SHM,2200 Watt,0,0,0,0,0,High
240,Rumah Dg LB 350 Di Gondokusuman 1 Km Dari St L...,"Rp 6,84 Miliar",6,3,447 m²,350 m²,0,SHM,2200 Watt,0,0,0,0,0,High
241,Rumah Bonus Tanah Dg LT 570 M2 Di Concat Untuk...,"Rp 4,5 Miliar",0,4,570 m²,278 m²,0,SHM,4400 Watt,0,0,0,0,0,High
242,Dijual Rumah Cantik Ngaglik Area Palagan & Kal...,"Rp 3,6 Miliar",0,0,133 m²,120 m²,0,SHM,0,0,0,0,0,0,High


In [ ]:
### Luxury 7B-20B

In [44]:
main_webUrl = "https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=20000000000&minPrice=7000000000&page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [45]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [46]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

55

In [47]:
dataframe_luxury = {}
dataframe_luxury["Nama"] = []
dataframe_luxury["Harga"] = []

In [48]:
for i in range(1,40):
    webUrl = f"https://www.rumah123.com/jual/daerah-istimewa-yogyakarta/rumah/?maxPrice=20000000000&minPrice=7000000000&page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe_luxury["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe_luxury["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue

        print(i)
        print(link, harga.text)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe_luxury.keys():
                dataframe_luxury[prop_key] = []
            
            if prop_key in dataframe_luxury.keys():
                dataframe_luxury[prop_key].append(prop_value)

            print(f'{prop_key}: {prop_value}')
            
        print("")


1
/properti/yogyakarta/hos19114328/ Rp 7,4 Miliar
Kamar Tidur: 5
Kamar Mandi: 5
Luas Tanah: 260 m²
Luas Bangunan: 600 m²
Carport: 4
Sertifikat: SHM
Daya Listrik: 5500 Watt
Kamar Tidur Pembantu: 2
Kamar Mandi Pembantu: 1
Garasi: 2
Jumlah Lantai: 3
Kondisi Properti: Bagus

1
/properti/yogyakarta/hos16449544/ Rp 9,7 Miliar
Kamar Tidur: 16
Kamar Mandi: 16
Luas Tanah: 962 m²
Luas Bangunan: 650 m²
Sertifikat: Lainnya
Daya Listrik: Lainnya

1
/properti/yogyakarta/hos19342734/ Rp 10 Miliar
Kamar Tidur: 7
Kamar Mandi: 5
Luas Tanah: 600 m²
Luas Bangunan: 749 m²
Carport: 5
Sertifikat: SHM
Daya Listrik: 3300 Watt
Kamar Tidur Pembantu: 2
Kamar Mandi Pembantu: 2
Garasi: 5
Jumlah Lantai: 1
Kondisi Properti: Bagus

1
/properti/yogyakarta/hos19350151/ Rp 7,5 Miliar
Kamar Tidur: 5
Kamar Mandi: 5
Luas Tanah: 200 m²
Luas Bangunan: 255 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 2
Jumlah Lantai: 2
Kondisi Properti: Bagus

1
/properti/yogyakarta/hos18330800/ Rp 19 Miliar
Kamar Tidur: 17
Kama

In [51]:
df_luxury = pd.DataFrame.from_dict(dataframe_luxury, orient='index').T
df_luxury = df_luxury.fillna(0)
df_luxury['Category'] = 'Luxury'
df_luxury.to_csv('Yogya_luxury.csv', index=False)

In [52]:
df_luxury

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Jumlah Lantai,Kondisi Properti,Category
0,Rumah Johar Luxury Brand New 3 Lantai Klasik M...,"Rp 7,4 Miliar",5,5,260 m²,600 m²,4,SHM,5500 Watt,2,1,2,3,Bagus,Luxury
1,"Guess house di Taman Siswa, Wirogunan Yogyakarta","Rp 9,7 Miliar",16,16,962 m²,650 m²,5,Lainnya,Lainnya,2,2,5,1,Bagus,Luxury
2,Rumah Mewah 2 Lantai Dengan Tanah Luas Di Pusa...,Rp 10 Miliar,7,5,600 m²,749 m²,2,SHM,3300 Watt,1,1,2,2,Bagus,Luxury
3,Rumah Mewah Tropis 2 Lantai Full Furnished Dal...,"Rp 7,5 Miliar",5,5,200 m²,255 m²,2,SHM,Lainnya,1,1,1,1,Bagus,Luxury
4,Rumah Shm Dijual Jogja Sagan Terban Gondokusum...,Rp 19 Miliar,17,7,895 m²,645 m²,1,SHM,3500 Watt,2,1,2,2,Bagus,Luxury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Dijual Rumah Hook 1 Lantai Yogyakarta 9821,"Rp 12,8 Miliar",0,0,698 m²,590 m²,0,SHM,8000 Watt,0,0,0,0,0,Luxury
214,"Rumah Caturtunggal, Yogyakarta 2 menit Kampus UGM","Rp 8,5 Miliar",0,0,513 m²,400 m²,0,SHM,0,0,0,0,0,0,Luxury
215,Rumah Klasik Di Pusat Kota Jogja,"Rp 8,5 Miliar",0,0,634 m²,400 m²,0,SHM,0,0,0,0,0,0,Luxury
216,Dijual Rumah Strategis Bisa Untuk Kost Sinduad...,"Rp 7,5 Miliar",0,0,500 m²,200 m²,0,SHM,0,0,0,0,0,0,Luxury
